In [90]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

In [2]:
output_notebook()

Loading BokehJS ...

In [51]:
import pandas as pd

from bokeh.models import LinearAxis, Range1d, LabelSet, ColumnDataSource
from bokeh.layouts import gridplot, row

In [11]:
from math import sqrt

In [37]:
%ls

test-lb-policy.csv   test.csv             visualization.ipynb


In [38]:
df = pd.read_csv('test.csv')
df_lb = pd.read_csv('test-lb-policy.csv')

In [39]:
df['std_dev'] = df.var_resp_time.apply(sqrt)

In [40]:
df['small_load'] = df.small_arrival / 18.73
df_lb['small_load'] = df.small_arrival / 18.73

In [56]:
p = figure(plot_width=400, plot_height=400, title="Mean response time")
source = ColumnDataSource(df)

p.line(df.small_load, df.avg_resp_time, line_color='navy')
p.line(df_lb.small_load, df_lb.avg_resp_time, line_color='navy',line_dash="4 4", legend='LB initial policy')
p.line(df_lb.small_load, 0.9*df_lb.avg_resp_time, line_color='gray',line_dash="4 4", legend='Constraint')
p.circle(df.small_load, df.avg_resp_time, color='white', line_color='navy',size=5)

labels = LabelSet(x="small_load", y="avg_resp_time", text="beta", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)
p.grid.grid_line_dash = [2,2]
p.grid.grid_line_alpha = 0.1

p.xaxis.axis_label = 'Small cell load'
p.legend.location  = 'bottom_right'

show(p) # show the results

In [52]:
p2 = figure(plot_width=400, plot_height=400, title="Average Power")

p2.line(df.small_load, df.avg_power, line_color='firebrick')
p2.line(df_lb.small_load, df_lb.avg_power, line_color='firebrick',line_dash="4 4", legend='LB initial policy')
p2.circle(df.small_load, df.avg_power, color='white', line_color='firebrick',size=5)

labels = LabelSet(x="small_load", y="avg_power", text="beta", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')

p2.add_layout(labels)
p2.grid.grid_line_dash = [2,2]
p2.grid.grid_line_alpha = 0.1

p2.xaxis.axis_label = 'Small cell load'
p2.legend.location  = 'bottom_right'

show(p2) # show the results

In [57]:
grid = gridplot([[p,p2]])

show(grid)

In [91]:
%ls

Four_small_cells_initial_plot.png  all_runs_3
all_ru-dispatch-decisions.txt      all_runs_4
all_run-dispatch-decisions.txt     all_runs_5
all_runlog                         all_runs_6
all_runs_0                         all_runs_7
all_runs_1                         all_runs_8
all_runs_10                        all_runs_9
all_runs_11                        all_runslog
all_runs_12                        result.csv
all_runs_13                        test-lb-policy.csv
all_runs_14                        test-test.csv
all_runs_15                        test.csv
all_runs_2                         visualization.ipynb


In [322]:
df=pd.read_csv('result.csv',header=None, skiprows=24)

In [323]:
df.head(5)

,0,1,2,3,4,5,6
0,0.08437,2.0,12.0,0.08,1.0,0.86994,1191.32
1,0.15256,2.0,12.0,0.17,1.0,0.90367,1139.71
2,0.00000,2.0,9.0,0.11,1.0,0.79583,1365.62
3,0.02429,2.0,9.0,0.00,1.0,0.81674,1334.67
4,0.04060,2.0,6.0,0.00,1.0,0.90261,1154.95


In [324]:
df.rename(columns={0:'beta', 1:'macro_arrival', 2:'small_arrival', 3:'avg_idle_time', 4:'avg_setup_time',5:'avg_resp_time',6:'avg_power'},inplace=True)

In [325]:
df.sort_values(by='small_arrival', inplace=True)

In [315]:
df[5:]

,beta,relative_idle_time,small_arrival,avg_resp_time,avg_power,small_load


In [330]:
import numpy as np
for i in range(28):
    if i <16:
        r=0
    else:
        r=1
    df_tmp = pd.read_csv('all_runs_'+str(i),skiprows=r,usecols=['beta','small_arrival','avg_idle_time','avg_power','avg_resp_time'],dtype={'beta':np.float64, 'small_arrival':np.float64,'avg_idle_time':np.float64,'avg_power':np.float64})
    df = df.append(df_tmp.loc[0])

In [331]:
df['relative_idle_time'] = df.small_arrival * df.avg_idle_time

In [332]:
df['relative_idle_time']=df.relative_idle_time.apply(round)

In [333]:
import copy
df = copy.copy(df[['beta','relative_idle_time','small_arrival','avg_resp_time','avg_power']])

In [334]:
df['small_load'] = df.small_arrival / 18.73
df.head(5)

,beta,relative_idle_time,small_arrival,avg_resp_time,avg_power,small_load
13,0.00000,0,4.0,0.75441,1281.31,0.213561
10,0.80785,2,4.0,0.69621,1073.76,0.213561
9,1.45908,6,4.0,0.46607,1065.65,0.213561
8,1.21848,4,4.0,0.55521,1068.75,0.213561
7,1.62559,8,4.0,0.40263,1063.25,0.213561


In [338]:
df['idx'] = list(range(56))
df.set_index('idx',inplace=True)

In [354]:
df.beta = df.beta.apply(lambda x: round(x, 4))

In [361]:

def filtered(arr, betazero=False):
    
    d = copy.copy(df)
    if betazero == False:
        #d = d[d['beta'] != 0.00000]
        d = d.loc[:27]
    else:
        #d = d[d['beta'] == 0.00000]
        d = d.loc[28:]
    d=d[d['small_arrival']==arr]
    d.sort_values(by='relative_idle_time',inplace=True)
    return d
p0 = figure(plot_width=600, plot_height=600, title="Mean response time")

source = ColumnDataSource(df.loc[0:27])

p0.line(filtered(4.0).relative_idle_time, filtered(4.0).avg_resp_time, line_color='navy',legend='Small cell load = 0.2')
p0.circle(filtered(4.0).relative_idle_time, filtered(4.0).avg_resp_time, color='white', line_color='navy',size=5)

p0.line(filtered(6.0).relative_idle_time, filtered(6.0).avg_resp_time, line_color='navy',line_dash="8 8", legend='                        = 0.32')
p0.circle(filtered(6.0).relative_idle_time, filtered(6.0).avg_resp_time, color='white', line_color='navy',size=5)

p0.line(filtered(9.0).relative_idle_time, filtered(9.0).avg_resp_time, line_color='navy',line_dash="4 4", legend='                        = 0.5')
p0.circle(filtered(9.0).relative_idle_time, filtered(9.0).avg_resp_time, color='white', line_color='navy',size=5)

p0.line(filtered(12.0).relative_idle_time, filtered(12.0).avg_resp_time, line_color='navy',line_dash="2 6", legend='                        = 0.65')
p0.circle(filtered(12.0).relative_idle_time, filtered(12.0).avg_resp_time, color='white', line_color='navy',size=5)

p0.line([0,1,2,10],[0.934192,0.934192,0.934192,0.934192],line_color='gray',legend='constraint')

#labels = LabelSet(x="relative_idle_time", y="avg_resp_time", text="beta", y_offset=8,
#                  text_font_size="8pt", text_color="#555555",
#                  source=source, text_align='center')
#p0.add_layout(labels)
p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time (relative to arrival rate)'
p0.legend.location  = 'top_right'

show(p0) # show the results

In [348]:
filtered(4)

,beta,relative_idle_time,small_arrival,avg_resp_time,avg_power,small_load
idx,,,,,,
0,0.00000,0,4.0,0.75441,1281.31,0.213561
6,0.50262,1,4.0,0.80249,1077.59,0.213561
1,0.80785,2,4.0,0.69621,1073.76,0.213561
3,1.21848,4,4.0,0.55521,1068.75,0.213561
2,1.45908,6,4.0,0.46607,1065.65,0.213561
4,1.62559,8,4.0,0.40263,1063.25,0.213561
5,1.75193,10,4.0,0.35863,1061.62,0.213561


In [360]:
p = figure(plot_width=600, plot_height=700, title="Power")
source = ColumnDataSource(df)

p.line(filtered(4.0).relative_idle_time, filtered(4.0).avg_power, line_color='navy',legend='Small cell load = 0.2')
p.circle(filtered(4.0).relative_idle_time, filtered(4.0).avg_power, color='white', line_color='navy',size=5)

p.line(filtered(6.0).relative_idle_time, filtered(6.0).avg_power, line_color='navy',line_dash="8 8", legend='                        = 0.32')
p.circle(filtered(6.0).relative_idle_time, filtered(6.0).avg_power, color='white', line_color='navy',size=5)

p.line(filtered(9.0).relative_idle_time, filtered(9.0).avg_power, line_color='navy',line_dash="4 4", legend='                        = 0.5')
p.circle(filtered(9.0).relative_idle_time, filtered(9.0).avg_power, color='white', line_color='navy',size=5)

p.line(filtered(12.0).relative_idle_time, filtered(12.0).avg_power, line_color='navy',line_dash="2 6", legend='                        = 0.65')
p.circle(filtered(12.0).relative_idle_time, filtered(12.0).avg_power, color='white', line_color='navy',size=5)

p.line(filtered(4.0,True).relative_idle_time, filtered(4.0,True).avg_power, line_color='firebrick')
p.circle(filtered(4.0, True).relative_idle_time, filtered(4.0,True).avg_power, color='white', line_color='firebrick',size=5)

p.line(filtered(6.0,True).relative_idle_time, filtered(6.0,True).avg_power, line_dash="8 8",line_color='firebrick')
p.circle(filtered(6.0, True).relative_idle_time, filtered(6.0,True).avg_power, color='white', line_color='firebrick',size=5)

p.line(filtered(9.0,True).relative_idle_time, filtered(9.0,True).avg_power, line_dash="4 4",line_color='firebrick')
p.circle(filtered(9.0, True).relative_idle_time, filtered(9.0,True).avg_power, color='white', line_color='firebrick',size=5)

p.line(filtered(12.0,True).relative_idle_time, filtered(12.0,True).avg_power, line_dash="2 6",line_color='firebrick')
p.circle(filtered(12.0, True).relative_idle_time, filtered(12.0,True).avg_power, color='white', line_color='firebrick',size=5)

#labels = LabelSet(x="relative_idle_time", y="avg_power", text="beta", y_offset=8,
#                  text_font_size="8pt", text_color="#555555",
#                  source=source, text_align='center')
#p.add_layout(labels)

p.grid.grid_line_dash = [2,2]
p.grid.grid_line_alpha = 0.1

p.xaxis.axis_label = 'idle time (relative to arrival rate)'
p.legend.location  = 'top_right'

show(p) # show the results

In [282]:
df

,beta,relative_idle_time,small_arrival,avg_resp_time,avg_power,small_load
13,0.00000,0,4.0,0.75441,1281.31,0.213561
10,0.80785,2,4.0,0.69621,1073.76,0.213561
9,1.45908,6,4.0,0.46607,1065.65,0.213561
8,1.21848,4,4.0,0.55521,1068.75,0.213561
7,1.62559,8,4.0,0.40263,1063.25,0.213561
6,1.75193,10,4.0,0.35863,1061.62,0.213561
11,0.50262,1,4.0,0.80249,1077.59,0.213561
4,0.04060,0,6.0,0.90261,1154.95,0.320342
12,1.48045,10,6.0,0.44747,1081.16,0.320342
14,1.34788,8,6.0,0.49293,1083.25,0.320342


In [182]:
df.loc[2,'avg_power'] = 1184

/Users/misikir/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [191]:
grid = gridplot([[p0,p]])

show(grid)

In [473]:
df=pd.read_csv('output_0.csv', skiprows=1)

In [474]:
df_rnd = df.loc[:0]
df = df.loc[20:]

In [475]:
for i in range(1,32):
    df_tmp = pd.read_csv('output_'+str(i)+'.csv', skiprows=1)
    df_rnd_tmp = df_tmp.loc[0,:]
    df_rnd = df_rnd.append(df_rnd_tmp)
    df_tmp = df_tmp.loc[20:]
    df = df.append(df_tmp)

In [476]:
df

,beta,macro_arrival,small_arrival,avg_idle_time,avg_setup_time,num_of_jobs,avg_resp_time,var_resp_time,avg_power
20,2.37319,2.0,1.0,0.00,1.0,3799648,0.18636,0.05252,878.49
20,2.36580,2.0,1.0,1.00,1.0,3799647,0.18625,0.05228,878.57
20,2.36779,2.0,1.0,2.00,1.0,3800885,0.18682,0.05285,878.73
20,2.38839,2.0,1.0,4.00,1.0,3799506,0.18611,0.05241,878.44
20,2.42418,2.0,1.0,8.00,1.0,3797351,0.18643,0.05241,878.49
20,2.49081,2.0,1.0,128.00,1.0,3797797,0.18704,0.05304,878.69
20,2.49684,2.0,1.0,512.00,1.0,3798646,0.18630,0.05237,878.58
20,2.49667,2.0,1.0,1024.00,1.0,3800222,0.18661,0.05250,878.58
20,0.03363,2.0,4.0,0.00,1.0,3798404,0.88806,1.07175,1111.50
20,0.39320,2.0,4.0,0.25,1.0,3799673,0.80045,1.10421,1077.78


In [477]:
df_rnd

,beta,macro_arrival,small_arrival,avg_idle_time,avg_setup_time,num_of_jobs,avg_resp_time,var_resp_time,avg_power
0,0.0,2.0,1.0,0.00,1.0,3799287.0,0.73024,0.91364,956.31
0,0.0,2.0,1.0,1.00,1.0,3796752.0,0.50912,0.71246,981.73
0,0.0,2.0,1.0,2.00,1.0,3800693.0,0.39707,0.57112,994.38
0,0.0,2.0,1.0,4.00,1.0,3801621.0,0.28731,0.40893,1007.14
0,0.0,2.0,1.0,8.00,1.0,3799785.0,0.19772,0.25976,1017.07
0,0.0,2.0,1.0,128.00,1.0,3799872.0,0.07454,0.02562,1030.92
0,0.0,2.0,1.0,512.00,1.0,3798899.0,0.06691,0.01034,1031.73
0,0.0,2.0,1.0,1024.00,1.0,3799074.0,0.06560,0.00745,1031.82
0,0.0,2.0,4.0,0.00,1.0,3798357.0,0.93703,1.18815,1094.80
0,0.0,2.0,4.0,0.25,1.0,3801422.0,0.79222,1.08885,1090.01


In [478]:
df['relative_idle_time'] = df.small_arrival * df.avg_idle_time
df.relative_idle_time    = df.relative_idle_time.apply(lambda x: round(x))

df_rnd['relative_idle_time'] = df_rnd.small_arrival * df_rnd.avg_idle_time
df_rnd.relative_idle_time    = df_rnd.relative_idle_time.apply(lambda x: round(x))

In [493]:
df_rel = pd.DataFrame()
df_rel['relative_idle_time'] = df.relative_idle_time.apply(lambda x: np.log2(x))
df_rel['small_arrival'] = df.small_arrival
df_rel['relative_power'] = [df.avg_power.iloc[i] / df_rnd.avg_power.iloc[i] for i in range(31)]

In [442]:
def logscale(arrival):
    rel_idle = df[df.small_arrival==arrival].relative_idle_time
    rel_idle = rel_idle.apply(lambda x: np.log2(x))
    return rel_idle

p0 = figure(plot_width=600, plot_height=600, title="Mean response time")

#source = ColumnDataSource(df.loc[0:27])

p0.line(logscale(1), df[df.small_arrival==1].avg_resp_time, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_resp_time, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_resp_time, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_resp_time, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_resp_time, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_resp_time, color='white', line_color='navy',size=5)



p0.line([0,1,2,10],0.859*0.95*np.ones(4),line_color='gray',legend='constraint')

p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. avarrival time)'
p0.legend.location  = 'top_right'

show(p0)

In [429]:
p0 = figure(plot_width=600, plot_height=600, title="Power")


p0.line(logscale(1), df[df.small_arrival==1].avg_power, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_power, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_power, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_power, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_power, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_power, color='white', line_color='navy',size=5)


p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. avarrival time)'
p0.legend.location  = 'top_right'

show(p0)

In [499]:
p0 = figure(plot_width=600, plot_height=600, title="Power")


p0.line(df_rel[df_rel.small_arrival==1].relative_idle_time, df_rel[df_rel.small_arrival==1].relative_power, line_color='navy',legend='Small cell load = 0.05')
p0.circle(df_rel[df_rel.small_arrival==1].relative_idle_time, df_rel[df_rel.small_arrival==1].relative_power, color='white', line_color='navy',size=5)

p0.line(df_rel[df_rel.small_arrival==4].relative_idle_time, df_rel[df_rel.small_arrival==4].relative_power, line_dash="15 2",line_color='navy',legend='Small cell load = 0.2')
p0.circle(df_rel[df_rel.small_arrival==4].relative_idle_time, df_rel[df_rel.small_arrival==4].relative_power, color='white', line_color='navy',size=5)

p0.line(df_rel[df_rel.small_arrival==6].relative_idle_time, df_rel[df_rel.small_arrival==6].relative_power, line_dash="4 4",line_color='navy',legend='Small cell load = 0.32')
p0.circle(df_rel[df_rel.small_arrival==6].relative_idle_time, df_rel[df_rel.small_arrival==6].relative_power, color='white', line_color='navy',size=5)

p0.line(df_rel[df_rel.small_arrival==9].relative_idle_time, df_rel[df_rel.small_arrival==9].relative_power, line_dash="1 4",line_color='navy',legend='Small cell load = 0.5')
p0.circle(df_rel[df_rel.small_arrival==9].relative_idle_time, df_rel[df_rel.small_arrival==9].relative_power, color='white', line_color='navy',size=5)

p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. avarrival time)'
p0.legend.location  = 'top_right'

show(p0)

# Idle power the same as busy power

In [430]:
df=pd.read_csv('output_32.csv', skiprows=1)

for i in range(33,64):
    df_tmp = pd.read_csv('output_'+str(i)+'.csv', skiprows=1)
    df_tmp = df_tmp.iloc[-1]
    df = df.append(df_tmp)

In [432]:
df['relative_idle_time'] = df.small_arrival * df.avg_idle_time
df.relative_idle_time    = df.relative_idle_time.apply(lambda x: round(x))

In [441]:
p0 = figure(plot_width=600, plot_height=600, title="Mean response time")

#source = ColumnDataSource(df.loc[0:27])

p0.line(logscale(1), df[df.small_arrival==1].avg_resp_time, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_resp_time, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_resp_time, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_resp_time, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_resp_time, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_resp_time, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_resp_time, color='white', line_color='navy',size=5)



p0.line([0,1,2,10],0.859*0.95*np.ones(4),line_color='gray',legend='constraint')

p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. avarrival time)'
p0.legend.location  = 'top_right'

show(p0)

In [435]:
p0 = figure(plot_width=600, plot_height=600, title="Power")


p0.line(logscale(1), df[df.small_arrival==1].avg_power, line_color='navy',legend='Small cell load = 0.05')
p0.circle(logscale(1), df[df.small_arrival==1].avg_power, color='white', line_color='navy',size=5)
p0.line(logscale(4), df[df.small_arrival==4].avg_power, line_color='navy',line_dash="15 2",legend='Small cell load = 0.2')
p0.circle(logscale(4), df[df.small_arrival==4].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(6), df[df.small_arrival==6].avg_power, line_color='navy',line_dash="4 4",legend='Small cell load = 0.32')
p0.circle(logscale(6), df[df.small_arrival==6].avg_power, color='white', line_color='navy',size=5)

p0.line(logscale(9), df[df.small_arrival==9].avg_power, line_color='navy',line_dash="1 4",legend='Small cell load = 0.5')
p0.circle(logscale(9), df[df.small_arrival==9].avg_power, color='white', line_color='navy',size=5)


p0.grid.grid_line_dash = [2,2]
p0.grid.grid_line_alpha = 0.1

p0.xaxis.axis_label = 'idle time in logscale (relative to avg. avarrival time)'
p0.legend.location  = 'bottom_right'

show(p0)